In [4]:
print("Sauvakävelyn data-analyysi - luokitteleva koneoppimismalli")

Sauvakävelyn data-analyysi - luokitteleva koneoppimismalli


In [5]:
import torch # Pytorch-neuroverkkokirjasto
import torch.nn as nn
import torch.optim as optim

# ---------------------------------------------------------
# 1. DATAN VALMISTELU
# ---------------------------------------------------------

# Luodaan "feikkitensorit", jolla voidaan testata mallia.
# Datan muoto on kuitenkin sama ([10, 26, 6]) kuin
# sauvakävelydatan tensoreissa, eli tämä malli toimii 
# heti myös oikealla datalla.

# Luodaan "Tehoton sauvakävely"-data (Luokka 0)
# Simuloidaan matalaa intensiteettiä (pienet arvot)
sk1_tensor = torch.randn(10, 26, 6) * 0.3 

# Luodaan "Tehokas sauvakävely"-data (Luokka 1)
# Simuloidaan korkeaa intensiteettiä (suuret arvot + siirtymä)
sk2_tensor = torch.randn(10, 26, 6) * 2.0 + 5.0

print(f"sk1-tensorin muoto: {sk1_tensor.shape}") # [10, 26, 6]
print(f"sk2-tensorin muoto: {sk2_tensor.shape}") # [10, 26, 6]

sk1-tensorin muoto: torch.Size([10, 26, 6])
sk2-tensorin muoto: torch.Size([10, 26, 6])


In [6]:
# ---------------------------------------------------------
# 2. HARJOITUSDATAN LUONTI
# ---------------------------------------------------------

# 1. Yhdistetään sk1- ja sk2-tensorit yhdeksi isoksi input-tensoriksi (X_train)
# Nyt meillä on 10 + 10 = 20 näytettä.
X_train = torch.cat((sk1_tensor, sk2_tensor), dim=0)

# 2. Luodaan vastaukset/luokat (y)
# Ensimmäiset 10 ovat nollia (sk1), seuraavat 10 ovat ykkösiä (sk2)
zeros = torch.zeros(10, dtype=torch.long)
ones = torch.ones(10, dtype=torch.long)
y_train = torch.cat((zeros, ones), dim=0)

# 3. Sekoitetaan data (Shuffle)
# Tämä tarvitaan, jotta malli ei opi järjestystä "ensin 10 nollaa, sitten 10 ykköstä"
indices = torch.randperm(len(X_train))
X_train = X_train[indices]
y_train = y_train[indices]

print(f"\nLopullinen opetusdata (X): {X_train.shape}") # [20, 26, 6]
print(f"Lopulliset luokat (y):     {y_train.shape}") # [20]


Lopullinen opetusdata (X): torch.Size([20, 26, 6])
Lopulliset luokat (y):     torch.Size([20])


In [7]:
# ---------------------------------------------------------
# 2. MALLIN MÄÄRITTELY (luokittelu vain kahteen luokkaan)
# ---------------------------------------------------------

class BinaryIMUClassifier(nn.Module):
    def __init__(self):
        super(BinaryIMUClassifier, self).__init__()
        
        # Input-kanavia on 6 (AccX, AccY, AccZ, GyroX, GyroY, GyroZ)
        self.conv1 = nn.Conv1d(in_channels=6, out_channels=16, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)
        
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        
        # AdaptiveAvgPool1d(1) tekee tensorista aina [Batch, 32, 1] riippumatta aika-akselin pituudesta (26, 30, 100...)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        
        # outputteja on 2 (tehoton sauvakävely vs tehokas sauvakävely)
        self.fc = nn.Linear(32, 2)

    def forward(self, x):
        # x: [Batch, Time, Channels] -> [20, 26, 6]
        
        # Käännetään kanavat oikeinpäin Conv1d:lle: [20, 6, 26]
        x = x.permute(0, 2, 1)
        
        x = self.pool(self.relu(self.conv1(x)))
        x = self.relu(self.conv2(x))
        
        x = self.global_pool(x)
        x = x.squeeze(-1) # [Batch, 32]
        
        x = self.fc(x) # [Batch, 2]
        return x

model = BinaryIMUClassifier()

In [10]:
# ---------------------------------------------------------
# 3. MALLIN HARJOITUS
# ---------------------------------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

print("\n--- Aloitetaan harjoitus ---")
for epoch in range(10): # 10 kierrosta riittää testaukseen
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    # Lasketaan tarkkuus
    predicted = torch.argmax(outputs, dim=1)
    accuracy = (predicted == y_train).float().mean()
    
    print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}, Accuracy = {accuracy.item()*100:.0f}%")


--- Aloitetaan harjoitus ---
Epoch 1: Loss = 0.1948, Accuracy = 100%
Epoch 2: Loss = 0.1731, Accuracy = 100%
Epoch 3: Loss = 0.1510, Accuracy = 100%
Epoch 4: Loss = 0.1276, Accuracy = 100%
Epoch 5: Loss = 0.1043, Accuracy = 100%
Epoch 6: Loss = 0.0820, Accuracy = 100%
Epoch 7: Loss = 0.0617, Accuracy = 100%
Epoch 8: Loss = 0.0443, Accuracy = 100%
Epoch 9: Loss = 0.0304, Accuracy = 100%
Epoch 10: Loss = 0.0200, Accuracy = 100%


In [11]:
# ---------------------------------------------------------
# 4. TESTAUS YHDELLÄ TENSORILLA
# ---------------------------------------------------------
print("\n--- Testataan yhdellä input-tensorilla ---")

# Luodaan yksi kahden sekunnin näyte, jonka muoto on [1, 26, 6]
# Näyte edustaa tehotonta sauvakävelyä
test_input = sk1_tensor[1].unsqueeze(0) # Lisätään batch-ulottuvuus: [26, 6] -> [1, 26, 6]

model.eval()
with torch.no_grad():
    prediction = model(test_input)
    probabilities = torch.nn.functional.softmax(prediction, dim=1)
    predicted_class = torch.argmax(probabilities).item()

classes = ["Tehoton sauvakävely (0)", "Tehokas sauvakävely (1)"]
print(f"Syötteen muoto: {test_input.shape}")
print(f"Mallin ennuste: {classes[predicted_class]}")
print(f"Todennäköisyydet: Tehoton sauvakävely {probabilities[0][0]:.2f}, Tehokas sauvakävely {probabilities[0][1]:.2f}")


--- Testataan yhdellä input-tensorilla ---
Syötteen muoto: torch.Size([1, 26, 6])
Mallin ennuste: Tehoton sauvakävely (0)
Todennäköisyydet: Tehoton sauvakävely 0.97, Tehokas sauvakävely 0.03
